*This file contains all code used for: creating dataset, adjusting dataset and training the model*

# Dataset Manipulating

### To create coords in Yolo format

In [1]:
#First thing to do is get the dataset in train_labes.csv and transform in YOLO training format
#Which is: <class_index> <x1> <y1> <x2> <y2> ...
#Where the coordinates were normalized, so:
  #X being the original bbox and x = X/img_width
  #Y being the original bbox and y = Y/img_heigth

import os
from PIL import Image

def get_image_dimensions(image_path):
    with Image.open(image_path) as img:
        # Get width and height
        width, height = img.size
        return width, height

#Getting to know the size images
folder_image_path = "C://Users//DELL/Desktop//Data_Driven_NASA_Challenge//spacecraft-pose-object-detection-runtime//data//images"
count = 0

for file in os.listdir(folder_image_path):
    img_path = os.path.join(folder_image_path, file)
    dimensions = get_image_dimensions(img_path)
    if dimensions != (1280,1024) :
        count +=1
        print(file,dimensions)
if count == 0:
    print("All ", len(os.listdir(folder_image_path))," images has (1280,1024) size ratio")


UnidentifiedImageError: cannot identify image file 'C://Users//DELL/Desktop//Data_Driven_NASA_Challenge//spacecraft-pose-object-detection-runtime//data//images\\data.yaml'

In [25]:
import pandas as pd

csv_path = "C://Users//DELL/Desktop//Data_Driven_NASA_Challenge//spacecraft-pose-object-detection-runtime//data//train_labels.csv"
df = pd.read_csv(csv_path)
print("Total txts: ",len(df))
df.head()


Total txts:  25801


,image_id,xmin,ymin,xmax,ymax
0,0001954c9f4a58f7ac05358b3cda8d20,0,277,345,709
1,00054819240f9d46378288b215dbcd3a,753,602,932,725
2,000dbf763348037b46558bbcb6a032ac,160,434,203,481
3,000e79208bebd8e84ce6c22fd8612a0d,70,534,211,586
4,000f13aff94499d03e3997afc55b0aa0,103,0,312,193


Before doing any manipulating it is necessary to see all images have annotations and vice versa

In [26]:
#Normalizing the coords
img_width = 1280
img_height = 1024
df["xmin_YOLO"] = df["xmin"]/img_width
df["ymin_YOLO"] = df["ymin"]/img_height
df["xmax_YOLO"] = df["xmax"]/img_width
df["ymax_YOLO"] = df["ymax"]/img_height
#df.head()

df_yolo = df.copy()
df_yolo.drop(["xmin", "ymin", "xmax", "ymax"], axis = 1, inplace= True)
df_yolo.head()

,image_id,xmin_YOLO,ymin_YOLO,xmax_YOLO,ymax_YOLO
0,0001954c9f4a58f7ac05358b3cda8d20,0.000000,0.270508,0.269531,0.692383
1,00054819240f9d46378288b215dbcd3a,0.588281,0.587891,0.728125,0.708008
2,000dbf763348037b46558bbcb6a032ac,0.125000,0.423828,0.158594,0.469727
3,000e79208bebd8e84ce6c22fd8612a0d,0.054688,0.521484,0.164844,0.572266
4,000f13aff94499d03e3997afc55b0aa0,0.080469,0.000000,0.243750,0.188477


### To create Txt files

In [27]:
#Creating the Training Txts
from tqdm import tqdm
import os

folder_txt_path = "C://Users//DELL/Desktop//Data_Driven_NASA_Challenge//spacecraft-pose-object-detection-runtime//data//training_folders//txts_labels_undivided"
for i, filename in tqdm(enumerate(df["image_id"])):
    output_file_path = os.path.join(folder_txt_path, filename+".txt")
    with open(output_file_path, 'w') as f:
        class_index = 0 #only 1 class: Sattelite
        xmin_YOLO = df["xmin_YOLO"][i]
        ymin_YOLO = df["ymin_YOLO"][i]
        xmax_YOLO = df["xmax_YOLO"][i]
        ymax_YOLO = df["ymax_YOLO"][i]
                        
                        
        line = f"{class_index} {xmin_YOLO} {ymin_YOLO} {xmin_YOLO} {ymax_YOLO} {xmax_YOLO} {ymax_YOLO} {xmax_YOLO} {ymin_YOLO}\n"
        f.write(line)

25801it [00:54, 475.02it/s] 


### Preparing folders

Before start the training, is necessary to set up the folder and the yaml file. For this, follow the structure bellow:

Yolo_training
|--file.yaml
|-- train
|   |-- images
|   |-- labels
|-- test
|   |-- images
|   |-- labels
|-- valid
|   |-- images
|   |-- labels


In [7]:
#Creating yaml file
from pathlib import Path

yaml_path = "C://Users//DELL/Desktop//Data_Driven_NASA_Challenge//spacecraft-pose-object-detection-runtime//data//data.yaml"
#/path/to/dataset/images/train
data_path = "C://Users//DELL/Desktop//Data_Driven_NASA_Challenge//spacecraft-pose-object-detection-runtime//data//training_folders"
train_path = "train//images"
#/path/to/dataset/images/val    
val_path =  "val//images"
#/path/to/dataset/images/test
test_path = "test//images"
nc = 1  # number of classes
names = ['Sat']

content = f"path: {data_path}\ntrain: {train_path}\nval:   {val_path}\ntest:  {test_path}\nnc:    {nc}\nnames: {names}\n"

with Path(yaml_path).open('w') as f:
    f.write(content)

## Splitting by type

In [28]:
#READING METADATA
import pandas as pd

csv_path = "C://Users//DELL/Desktop//Data_Driven_NASA_Challenge//spacecraft-pose-object-detection-runtime//data//train_metadata.csv"
df = pd.read_csv(csv_path)
print("Total txts: ",len(df))
df.head()


Total txts:  25801


,image_id,spacecraft_id,background_id
0,0001954c9f4a58f7ac05358b3cda8d20,24,247
1,00054819240f9d46378288b215dbcd3a,14,10
2,000dbf763348037b46558bbcb6a032ac,19,17
3,000e79208bebd8e84ce6c22fd8612a0d,14,15
4,000f13aff94499d03e3997afc55b0aa0,28,15


In [29]:
#CREATING LIST OF NAMES BY TYPE
train_imgs_files =[]
val_imgs_files = []
test_imgs_files = []
val_split = 0.1

types_spc = list(set(df["spacecraft_id"]))
types_back = list(set(df["background_id"]))

for spc in types_spc:
    filt_df = df[df["spacecraft_id"] == spc]
    count = 0
    for img in filt_df["image_id"]:
        if count <= (1-2*val_split)*len(filt_df): #train
            train_imgs_files.append(img)
        elif (1-2*val_split)*len(filt_df) < count < (1-val_split)*len(filt_df): #val
            val_imgs_files.append(img)
        elif val_split*len(filt_df) <= count: #test
            test_imgs_files.append(img)
        count += 1

In [30]:
print(len(train_imgs_files), len(val_imgs_files), len(test_imgs_files) )

20652 2576 2573


In [31]:
#TRAIN DATA BY TYPE
tdf = df[df['image_id'].isin(train_imgs_files)]
tdf["spacecraft_id"].value_counts().sort_index()

spacecraft_id
2     1030
3     1184
5     1213
6     1328
11    1265
13     959
14    1381
18    1477
19    1421
20    1467
22    1313
24    1677
25    1644
28    1683
30    1610
Name: count, dtype: int64

In [32]:
#VAL DATA BY TYPE
tdf = df[df['image_id'].isin(val_imgs_files)]
tdf["spacecraft_id"].value_counts().sort_index()

spacecraft_id
2     129
3     148
5     151
6     166
11    157
13    120
14    172
18    184
19    177
20    183
22    163
24    210
25    205
28    210
30    201
Name: count, dtype: int64

In [33]:
#TEST DATA BY TYPE
tdf = df[df['image_id'].isin(test_imgs_files)]
tdf["spacecraft_id"].value_counts().sort_index()

spacecraft_id
2     128
3     147
5     151
6     165
11    158
13    119
14    172
18    184
19    177
20    183
22    164
24    209
25    205
28    210
30    201
Name: count, dtype: int64

In [34]:
#MOVING THE TXT FILES
#paths to txt folders
import os 
from tqdm import tqdm
import shutil
    
source_folder =  'C://Users//DELL/Desktop//Data_Driven_NASA_Challenge//spacecraft-pose-object-detection-runtime//data//training_folders//txts_labels_undivided'
train_labels_destination_folder = "C://Users//DELL/Desktop//Data_Driven_NASA_Challenge//spacecraft-pose-object-detection-runtime//data//training_folders//train//labels"
val_labels_destination_folder = "C://Users//DELL/Desktop//Data_Driven_NASA_Challenge//spacecraft-pose-object-detection-runtime//data//training_folders//val//labels"
test_labels_destination_folder = "C://Users//DELL/Desktop//Data_Driven_NASA_Challenge//spacecraft-pose-object-detection-runtime//data//training_folders//test//labels"

for file in tqdm(train_imgs_files):
    source = os.path.join(source_folder, file+ ".txt")
    destination_path = os.path.join(train_labels_destination_folder, file+".txt")
    shutil.move(source,destination_path)
for file in tqdm(val_imgs_files):
    source = os.path.join(source_folder, file+ ".txt")
    destination_path = os.path.join(val_labels_destination_folder, file+".txt")
    shutil.move(source,destination_path)
for file in tqdm(test_imgs_files):
    source = os.path.join(source_folder, file+ ".txt")
    destination_path = os.path.join(test_labels_destination_folder, file+".txt")
    shutil.move(source,destination_path)

100%|██████████████████████████████████████████████████████████████████████████████| 2573/2573 [00:48<00:00, 52.63it/s]


In [13]:
#MOVING THE IMAGES
#paths to txt folders
import os 
from tqdm import tqdm
import shutil
    
source_folder =  'C://Users//DELL/Desktop//Data_Driven_NASA_Challenge//spacecraft-pose-object-detection-runtime//data//images'
train_labels_destination_folder = "C://Users//DELL/Desktop//Data_Driven_NASA_Challenge//spacecraft-pose-object-detection-runtime//data//training_folders//train//images"
val_labels_destination_folder = "C://Users//DELL/Desktop//Data_Driven_NASA_Challenge//spacecraft-pose-object-detection-runtime//data//training_folders//val//images"
test_labels_destination_folder = "C://Users//DELL/Desktop//Data_Driven_NASA_Challenge//spacecraft-pose-object-detection-runtime//data//training_folders//test//images"

for file in tqdm(train_imgs_files):
    source = os.path.join(source_folder, file+ ".png")
    destination_path = os.path.join(train_labels_destination_folder, file+".png")
    shutil.move(source,destination_path)
for file in tqdm(val_imgs_files):
    source = os.path.join(source_folder, file+ ".png")
    destination_path = os.path.join(val_labels_destination_folder, file+".png")
    shutil.move(source,destination_path)
for file in tqdm(test_imgs_files):
    source = os.path.join(source_folder, file+ ".png")
    destination_path = os.path.join(test_labels_destination_folder, file+".png")
    shutil.move(source,destination_path)

100%|█████████████████████████████████████████████████████████████████████████████| 2573/2573 [00:07<00:00, 358.51it/s]


## TRAINING WITH THE SPLITTING DATA BY TYPE


In [2]:
#Loading pre-trained model
from ultralytics import YOLO
model = YOLO("C://Users//DELL//Desktop//Data_Driven_NASA_Challenge//spacecraft-pose-object-detection-runtime//data//runs//detect//train11/weights//best.pt")

C:\Users\DELL\anaconda3\lib\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
C:\Users\DELL\anaconda3\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.4' currently installed).
  from pandas.core import (


In [ ]:
# Train the model
model.train(data="data.yaml", epochs=20, imgsz=640, single_cls = True, batch = 64)

New https://pypi.org/project/ultralytics/8.2.8 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.1.11 🚀 Python-3.9.12 torch-2.2.1+cpu CPU (Intel Core(TM) i7-7500U 2.70GHz)
engine\trainer: task=detect, mode=train, model=C://Users//DELL//Desktop//Data_Driven_NASA_Challenge//spacecraft-pose-object-detection-runtime//data//runs//detect//train11/weights//best.pt, data=data.yaml, epochs=20, time=None, patience=50, batch=64, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train12, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=True, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False

train: Scanning C:\Users\DELL\Desktop\Data_Driven_NASA_Challenge\spacecraft-pose-object-detection-runtime\data\training
val: Scanning C:\Users\DELL\Desktop\Data_Driven_NASA_Challenge\spacecraft-pose-object-detection-runtime\data\training_f


Plotting labels to runs\detect\train12\labels.jpg... 
"No such keys(s): 'mode.use_inf_as_null'"
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to runs\detect\train12
Starting training for 20 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       1/20         0G     0.7265     0.6017     0.9952         68        640: 100%|██████████| 323/323 [18:41:24<00:00
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 21/21 [15:08

                   all       2576       2576      0.946      0.881       0.95      0.803



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       2/20         0G     0.7781     0.6468       1.02        120        640:  59%|█████▉    | 192/323 [4:48:40<3:16:5
